In [162]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv("data/imputed_dataset.csv")

In [163]:
df = df[(df['fyear'] >= 2000) & (df['fyear'] <= 2023)]
df = df.dropna(subset=['prev_inv'])

In [164]:
X = df[['liquidity', 'leverage', 'roa', 'icapt', 'aqc', 'prev_inv', 'cpu_index', 'asset_growth']]
y = df['y']

In [165]:
X = X.applymap(lambda x: np.log(x) if x > 0 else x)
y = np.log(df['y'])

In [166]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [167]:
X.head()

,liquidity,leverage,roa,icapt,aqc,prev_inv,cpu_index,asset_growth
276,-0.405825,-0.693197,-3.163223,8.375122,0.000000,-2.037374,4.134346,0.078636
277,0.540221,-0.690331,-1.702159,9.174431,0.000000,-2.904557,4.134346,0.054608
278,0.775244,-0.952044,-1.769399,8.375547,0.000000,-2.118500,4.134346,0.275756
279,0.272252,-0.718890,-4.198557,8.622022,5.138149,-2.434787,4.134346,0.004241
280,0.390569,-0.296824,-2.719636,9.521348,7.833204,-3.319816,4.134346,0.067703


In [168]:
X.isnull().sum()

liquidity       0
leverage        0
roa             0
icapt           0
aqc             0
prev_inv        0
cpu_index       0
asset_growth    0
dtype: int64

# Model 1 without rnd

In [169]:
import statsmodels.api as sm

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     2116.
Date:                Fri, 08 Mar 2024   Prob (F-statistic):               0.00
Time:                        13:20:15   Log-Likelihood:                -3569.4
No. Observations:                6347   AIC:                             7157.
Df Residuals:                    6338   BIC:                             7218.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.4519      0.068      6.633   

In [170]:
# Check instrument relevance (F-statistic)
f_statistic = model.fvalue
if f_statistic < 10:
    print("Warning: Instruments may not be relevant.")
else: 
    print("Significant Instruments")

Significant Instruments


**t-values**:
   - t-values represent the ratio of the estimated coefficient to its standard error.
   - They indicate the significance of the estimated coefficient.
   - Larger absolute t-values (in either direction) suggest greater evidence against the null hypothesis that the coefficient is zero.
   
**P-values (P>|t|)**:
   - P-values represent the probability of observing the estimated coefficient (or more extreme) if the null hypothesis (that the coefficient is zero) is true.
   - Small p-values (typically less than a chosen significance level, such as 0.05) suggest that the coefficient is statistically significant.
   - In this output, all p-values are very close to zero (0.000), indicating that all coefficients are statistically significant at conventional significance levels (e.g., 0.05).
   
**Confidence Intervals ([0.025 0.975])**:
   - Confidence intervals provide a range of plausible values for the true population parameter (the coefficient) based on the sample data.
   - They are typically constructed around the estimated coefficient and provide a sense of the precision of the estimate.
   - The confidence interval is calculated as the estimated coefficient plus or minus a margin of error.
   - In this output, the confidence intervals indicate the range within which we are reasonably confident the true coefficients lie.

**R-squared (or Adjusted R-squared) Test:**
R-squared measures the proportion of variance in the dependent variable that is explained by the independent variables in the model.
Adjusted R-squared adjusts for the number of independent variables in the model, providing a more conservative measure of model fit.
Both R-squared and Adjusted R-squared range from 0 to 1, with higher values indicating better fit.

In [171]:
# Calculate R-squared
r_squared = model.rsquared
print("R-squared:", r_squared)

# Calculate Adjusted R-squared
adjusted_r_squared = model.rsquared_adj
print("Adjusted R-squared:", adjusted_r_squared)


R-squared: 0.7275620320910468
Adjusted R-squared: 0.7272181533054248


The F-test assesses the overall significance of the model by comparing the explained variance to the unexplained variance.
A significant F-test indicates that at least one independent variable has a nonzero coefficient, suggesting that the model as a whole is significant.

In [172]:
# Get F-statistic and associated p-value
f_statistic = model.fvalue
p_value = model.f_pvalue

print("F-statistic:", f_statistic)
print("p-value:", p_value)


F-statistic: 2115.7514290253553
p-value: 0.0


# Model 2 with R&D

In [173]:
df['xrd'].fillna(0, inplace=True)
X2 = df[['liquidity', 'leverage', 'roa', 'icapt', 'aqc', 'prev_inv', 'cpu_index', 'asset_growth', 'xrd']]

In [174]:
X2 = sm.add_constant(X2)

model2 = sm.OLS(y, X2).fit()

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     881.2
Date:                Fri, 08 Mar 2024   Prob (F-statistic):               0.00
Time:                        13:20:15   Log-Likelihood:                -5120.4
No. Observations:                6347   AIC:                         1.026e+04
Df Residuals:                    6337   BIC:                         1.033e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -3.6190      0.024   -147.926   

# Model 3 with change in investment

In [175]:
df['y'] = df['y'] - df['prev_inv']
X3 = df[['liquidity', 'leverage', 'roa', 'icapt', 'aqc', 'cpu_index', 'asset_growth']]
y = df['y']

In [176]:
X3 = sm.add_constant(X3)

model3 = sm.OLS(y, X3).fit()

print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     24.65
Date:                Fri, 08 Mar 2024   Prob (F-statistic):           2.13e-33
Time:                        13:20:15   Log-Likelihood:                 14494.
No. Observations:                6347   AIC:                        -2.897e+04
Df Residuals:                    6339   BIC:                        -2.892e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0111      0.001     10.963   